In [5]:
import ClassiferHelperAPI as CH
import importlib
import pandas as pd
import csv
import json
importlib.reload(CH)
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics

In [2]:
# Remove unwanted columns
df = pd.DataFrame.from_csv("../FinalResults/ImgShrRnkListWithTags.csv")
cols = list(df.columns)
df.drop('URL',1,inplace=True)
df.drop('Album',1,inplace=True)
# df.drop('INDIVIDUAL_NAME',1,inplace=True)
df.reset_index(inplace=True)
df = df.iloc[np.random.permutation(len(df))]
df.to_csv("/tmp/test.csv",index=False)

In [3]:
reader = csv.reader(open("/tmp/test.csv","r"))
head = reader.__next__()
data = {}
for row in reader:
    temp = {}
    for i in range(1,len(row)):
        temp[head[i]] = row[i] 
    data[row[0]] = temp
    
json.dump(data,open("/tmp/test.json","w"),indent=4)
allAttribs = CH.genHead(data,'SPECIES') + CH.genHead(data,'SEX') + CH.genHead(data,'AGE') + CH.genHead(data,'QUALITY') + CH.genHead(data,'VIEW_POINT') + CH.genHead(data,'tags')

In [19]:
def getAccuracyDict(thres):    
    importlib.reload(CH)
    accuracy_dict  = {}
    for i in np.arange(0.1,1,0.1):
        temp = {}
        temp["Linear"] = CH.buildRegrModel(data,allAttribs,i,thres,False)
        temp["Logistic"] = CH.buildRegrModel(data,allAttribs,i,thres)

        accuracy_dict[i*100] = temp
        
    return accuracy_dict


In [23]:
# THRESHOLD = 5
df1 = pd.DataFrame(getAccuracyDict(5)).transpose() 

df1.Logistic

10    85.521236
20    85.124864
30    86.600496
40    87.554269
50    87.673611
60    87.201735
70    88.150289
80    87.445887
90    86.206897
Name: Logistic, dtype: float64

In [6]:
# THRESHOLD = 10
df2 = pd.DataFrame(getAccuracyDict(10)).transpose() 
df2.plot()
df2

,Linear,Logistic
10,20.366795,55.212355
20,22.366992,58.523344
30,26.426799,56.327543
40,23.444284,58.610709
50,27.256944,58.333333
60,31.236443,60.737527
70,29.768786,58.670520
80,32.900433,58.441558
90,27.586207,65.517241


In [7]:
# THRESHOLD = 15
df3 = pd.DataFrame(getAccuracyDict(15)).transpose() 
df3.plot()
df3

,Linear,Logistic
10,32.046332,55.212355
20,33.550489,58.523344
30,40.694789,56.327543
40,34.732272,58.610709
50,40.625000,58.333333
60,42.299349,60.737527
70,43.641618,58.670520
80,42.424242,58.441558
90,42.241379,65.517241


In [8]:
# THRESHOLD = 20
df4 = pd.DataFrame(getAccuracyDict(20)).transpose() 
df4.plot()
df4

,Linear,Logistic
10,42.567568,55.212355
20,43.865364,58.523344
30,50.372208,56.327543
40,46.743849,58.610709
50,54.687500,58.333333
60,57.700651,60.737527
70,55.202312,58.670520
80,53.246753,58.441558
90,54.310345,65.517241


In [9]:
# THRESHOLD = 25
df5 = pd.DataFrame(getAccuracyDict(25)).transpose() 
df5.plot()
df5

,Linear,Logistic
10,50.482625,55.212355
20,52.768730,58.523344
30,61.786600,56.327543
40,58.321274,58.610709
50,64.756944,58.333333
60,66.377440,60.737527
70,68.208092,58.670520
80,64.935065,58.441558
90,64.655172,65.517241


In [10]:
df1.plot()
plt.title("Threshold = 5")
plt.xlabel("Train-Test Split Percentage")
plt.ylabel("Accuracy")
df2.plot()
plt.title("Threshold = 10")
plt.xlabel("Train-Test Split Percentage")
plt.ylabel("Accuracy")
df3.plot()
plt.title("Threshold = 15")
plt.xlabel("Train-Test Split Percentage")
plt.ylabel("Accuracy")
df4.plot()
plt.title("Threshold = 20")
plt.xlabel("Train-Test Split Percentage")
plt.ylabel("Accuracy")
df5.plot()
plt.title("Threshold = 25")
plt.xlabel("Train-Test Split Percentage")
plt.ylabel("Accuracy")
plt.show()

In [5]:
from sklearn.linear_model import LogisticRegression, LinearRegression
gidAttribDict = CH.createDataFlDict(data,allAttribs)
splitProp = 0.5
nTrain = int(len(gidAttribDict.keys()) * splitProp)
nTest = len(gidAttribDict.keys()) - int(len(gidAttribDict.keys()) * splitProp)

df = pd.DataFrame(gidAttribDict).transpose()
df = df[allAttribs + ["TARGET"]] # Rearranging the order of the columns

attributes = df.columns[:len(allAttribs)] # all attributes

train_x = df[attributes].head(nTrain)
train_y = df['TARGET'].head(nTrain)
regr = LogisticRegression()
regr.fit(train_x,train_y)

test_x = df[attributes].tail(nTest)
test_y =df['TARGET'].tail(nTest) 
test_y = list(test_y)

predictions = regr.predict(test_x)
predictions = list(predictions)

In [14]:
from sklearn import metrics
#from ggplot import *

preds = regr.predict_proba(test_x)[:,1]

fpr, tpr, _ = metrics.roc_curve(test_y, preds)
auc = metrics.auc(fpr,tpr)
print(auc)
df = pd.DataFrame(dict(fpr=fpr, tpr=tpr))
# ggplot(df, aes(x='fpr', y='tpr')) +\
#     geom_line() +\
#     geom_abline(linetype='dashed')

0.706817626953


In [15]:
plt.plot(fpr,tpr)
plt.plot([0,0.5,1],[0,0.5,1])
plt.show()

In [6]:
aucDict = {}
for i in np.arange(0.1,1,0.1):
    test_x, test_y, preds, _ = CH.buildRegrModel(data,allAttribs,i)
    fpr, tpr, _ = metrics.roc_curve(test_y, preds)
    auc = metrics.auc(fpr,tpr)
    title = str("AUC for train-test split of %f percent = %f" %(i*100,auc))
    plt.plot(fpr,tpr,label=int(i*100))
    plt.title("ROC Curve")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    aucDict[int(i*100)] = auc
    print(title)

AUC for train-test split of 10.000000 percent = 0.634999
AUC for train-test split of 20.000000 percent = 0.673990
AUC for train-test split of 30.000000 percent = 0.640692
AUC for train-test split of 40.000000 percent = 0.657440
AUC for train-test split of 50.000000 percent = 0.706818
AUC for train-test split of 60.000000 percent = 0.701304
AUC for train-test split of 70.000000 percent = 0.715686
AUC for train-test split of 80.000000 percent = 0.722041
AUC for train-test split of 90.000000 percent = 0.805000


In [7]:
plt.plot([0,0.5,1],[0,0.5,1])
plt.legend()
plt.show()

In [52]:
df = pd.DataFrame(aucDict,index=['AUC']).transpose()
df.reset_index(inplace=True)
df.columns = ['Split Percent','AUC']
df

,Split Percent,AUC
0,10,0.634999
1,20,0.673990
2,30,0.640692
3,40,0.657440
4,50,0.706818
5,60,0.701304
6,70,0.715686
7,80,0.722041
8,90,0.805000
